# Use pyquery-ql.py with files
Send a graphql query to GitHub
and work with files for reports.

Supports Python 3.6+

In [ ]:
import csv
import json
import os
import pprint

import requests

In [ ]:
# get api token and set authorization
api_token = os.environ['GITHUB_API_TOKEN']
headers = {'Authorization': f'token {api_token}'}

In [ ]:
# set url to a graphql endpoint
url = 'https://api.github.com/graphql'

In [ ]:
# add a json query
query = """
{
  organization(login: "jupyterhub") {
    repositories(first: 30) {
      totalCount
      edges {
        node {
          name
          url
          issues(states: OPEN) {
            totalCount
          }
          pullRequests(states: OPEN) {
            totalCount
          }
        }
      }
    }
  }
}
"""

### Make request and create json and csv files

In [ ]:
# submit the request
r = requests.post(url=url, json={'query': query}, headers=headers)

In [ ]:
# create a json file from response
with open('data.json', 'w') as f:
    json.dump(r.json(), f)

In [ ]:
# unpack the layers of json
nodes = r.json()['data']['organization']['repositories']['edges']

unpacked = []
for node in nodes:
    unpacked.append(node['node'])

In [ ]:
headers = ['name', 'url', 'issues', 'prs']

rows = []
for obj in unpacked:
    new_dict = {'name':obj['name'], 'url':obj['url'], 'issues':obj['issues']['totalCount'], 'prs':obj['pullRequests']['totalCount']}
    rows.append(new_dict)

In [ ]:
with open('mydata.csv', 'w') as f:
    f_csv = csv.DictWriter(f, headers)
    f_csv.writeheader()
    f_csv.writerows(rows)

Check file

In [ ]:
%%bash

less mydata.csv

## Bring into pandas

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('mydata.csv')

In [ ]:
df.columns

In [ ]:
# df.head()

Generate basic report of total open issues

In [ ]:
# df.dtypes

In [ ]:
# df.index

In [ ]:
# df.values

### Reports

In [ ]:
# By repo name
df.sort_values(by=['name'])

In [ ]:
# By open issue count
df.sort_values(by=['issues'], ascending=False)

In [ ]:
# by open pr count
df.sort_values(by=['prs'], ascending=False)

In [ ]:
# output data to a csv
# df.to_csv('issue_report.csv')